<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Langchain_RAG_Implementaion_using_Mistral_AI_and__Embed_Caching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain Faiss-gpu tiktoken sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gdwls0dp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gdwls0dp
  Resolved https://github.com/huggingface/transformers to commit 1b8decb04c246ec8e1c4ba7f2749043d0876d24e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.34.0.dev0-py3-none-any.whl size=7745781 sha256=9bd76972bcca228aaeded312d3ec5b3dfd37b05735861f2bda95a13bf41f8396
  Stored in directory: /tmp/pip-ephem-wheel-cache-o_vm1x8u/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing 

In [3]:
# Base ctransformers with CUDA GPU acceleration
! pip install ctransformers[cuda]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 845.8/845.8 kB 50.0 MB/s eta 0:00:00


## Load Dataset

In [4]:
!wget https://raw.githubusercontent.com/AI-Maker-space/DataRepository/main/oppenheimer.csv

--2023-10-02 14:44:38--  https://raw.githubusercontent.com/AI-Maker-space/DataRepository/main/oppenheimer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80957 (79K) [text/plain]
Saving to: ‘oppenheimer.csv’

oppenheimer.csv     100%[===================>]  79.06K  --.-KB/s    in 0.02s   

2023-10-02 14:44:38 (3.39 MB/s) - ‘oppenheimer.csv’ saved [80957/80957]



In [5]:
!wget https://raw.githubusercontent.com/AI-Maker-space/DataRepository/main/barbie.csv

--2023-10-02 14:45:02--  https://raw.githubusercontent.com/AI-Maker-space/DataRepository/main/barbie.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72289 (71K) [text/plain]
Saving to: ‘barbie.csv’

barbie.csv          100%[===================>]  70.59K  --.-KB/s    in 0.02s   

2023-10-02 14:45:02 (2.98 MB/s) - ‘barbie.csv’ saved [72289/72289]



## Data Parsing using langchain

In [8]:
from langchain.document_loaders.csv_loader import CSVLoader
#
barbie_loader = CSVLoader("/content/barbie.csv").load()
print(f"len of documents in barbie documents :{len(barbie_loader)}")
#
oppenheimer_loader = CSVLoader("/content/oppenheimer.csv").load()
print(f"len of documents in oppenheimer documents :{len(oppenheimer_loader)}")

len of documents in barbie documents :125
len of documents in oppenheimer documents :150


## Create Managebale pieces of text by using RecursiveCharacterTextSplitter to create chunks for reviews

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400,
                                      chunk_overlap=10,
                                      length_function=len,)
#
barbie_documents = text_splitter.transform_documents(barbie_loader)
print(f"number of chunks in barbie documents : {len(barbie_documents)}")
#
oppenheimer_documents = text_splitter.transform_documents(oppenheimer_loader)
print(f"number of chunks in oppenheimer documents : {len(oppenheimer_documents)}")

number of chunks in barbie documents : 354
number of chunks in oppenheimer documents : 414


In [29]:
combined_documents = barbie_documents + oppenheimer_documents

In [30]:
len(combined_documents)

768

## Create Vectorstore

* Here we will leverage a CacheBackedEmbeddings to prevent us from reembedding similar queries over and over again.
* Structured documents will be passed into a usesful format for querying ,retrieving and use in LLM application
* here we will use FAISS(Facebook AI similarity search) as the vectorstore.

In [12]:
from langchain.embeddings import CacheBackedEmbeddings,HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

In [31]:
store = LocalFileStore("./cache/")
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
core_embeddings_model = HuggingFaceEmbeddings(model_name=embed_model_id)
embedder = CacheBackedEmbeddings.from_bytes_store(core_embeddings_model,
                                                  store,
                                                  namespace=embed_model_id)

## Create Vectorstore

In [32]:
vectorstore = FAISS.from_documents(combined_documents,embedder)

## Embed Query

In [33]:
query = "How is Will Ferrell in this movie?"
embedding_vector = core_embeddings_model.embed_query(query)
len(embedding_vector)

384

## Retrieve passages from the vectorstore that are similar to the query

In [34]:
docs = vectorstore.similarity_search_by_vector(embedding_vector,k=10)

In [35]:
for page in docs:
  print(page.page_content)

Review: I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow. The film was very disjointed. Ryan Gosling as good as he is seemed to old to play Ken and Will Ferrell ruined every scene he was in. I just didn't get it, it seemed hollow artificial and hackneyed. A waste of some great talent. It was predictable without being
Review: The first half was pretty enjoyable, fun, light, but it took itself too seriously by the second half. No longer allowing the talented cast, especially Gosling, to shine and make us laugh. It felt like the talents of Will Ferrell and Michael Cera were also somewhat underutilized. Interesting concept, had potential, but later in the movie, it definitely started to fall flat for me.
: 30
Review_Date: 21 July 2023
Author: imdbmovieguy
Rating: 
Review_Title: Honestly - not funny and light enough
: 78
Review_Date: 20 July 2023
Author: moriumemoltre
Rating: 6
Review_Title: Great acting but why th

## Check to see  how much time CacheBackedEmbeddings pattern saves us

In [36]:
%%timeit -n 2 -r 2
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
#
embedding_vector = core_embeddings_model.embed_query(query)
docs = vectorstore.similarity_search_by_vector(embedding_vector,k=4)


10.4 ms ± 2.78 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


In [37]:
%%timeit -n 1 -r 1
query = "How is Will Ferrell in this movie?"
#
embedding_vector = core_embeddings_model.embed_query(query)
docs = vectorstore.similarity_search_by_vector(embedding_vector,k=4)

12.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [38]:
%%timeit -n 2 -r 2
query = "How is Will Ferrell in this movie?"
#
embedding_vector = core_embeddings_model.embed_query(query)
docs = vectorstore.similarity_search_by_vector(embedding_vector,k=4)

8.53 ms ± 1.49 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


## Building a retrieval chain

In [24]:
from langchain.llms import CTransformers
config = {'max_new_tokens': 256, 'repetition_penalty': 1.1,'temperature':0.9}
llm = CTransformers(model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
                    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",config=config)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
#
handler = StdOutCallbackHandler()
#
qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = vectorstore.as_retriever(),
    callbacks=[handler],
    return_source_documents=True
)

## Provide the user query

In [40]:
%%time
query = "How was Will Ferrell in this movie?"
qa_with_sources_chain({"query":query})



> Entering new RetrievalQA chain...

> Finished chain.
CPU times: user 6min 47s, sys: 904 ms, total: 6min 48s
Wall time: 4min 9s


{'query': 'How was Will Ferrell in this movie?',
 'result': ' Will Ferrell ruined every scene he was in.',
 'source_documents': [Document(page_content="Review: I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow. The film was very disjointed. Ryan Gosling as good as he is seemed to old to play Ken and Will Ferrell ruined every scene he was in. I just didn't get it, it seemed hollow artificial and hackneyed. A waste of some great talent. It was predictable without being", metadata={'source': '/content/barbie.csv', 'row': 61}),
  Document(page_content='Review: The first half was pretty enjoyable, fun, light, but it took itself too seriously by the second half. No longer allowing the talented cast, especially Gosling, to shine and make us laugh. It felt like the talents of Will Ferrell and Michael Cera were also somewhat underutilized. Interesting concept, had potential, but later in the movie, it definitely started

In [41]:
query = "Did these movies explore themes of existentialism?"
qa_with_sources_chain({"query":query})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Did these movies explore themes of existentialism?',
 'result': ' Yes, both movies explored themes of existentialism. The Imitation Game focused heavily on the psychological of those involved in the war effort and how their actions shaped the world, while Barbie in a Pink Wig also delved into the psychological of individuals and how they perceive reality.',
 'source_documents': [Document(page_content='the real world: its like everyone see that reality is not really what they perceived themselves in their own reality and power will shift also later in Barbie Land. I understand why they act like this and stuff but i didnt really get into the movie to be honest: i only identify myself when they talked about society and business but all the musical things, the choregraphy and even the morality was', metadata={'source': '/content/barbie.csv', 'row': 36}),
  Document(page_content='the real world: its like everyone see that reality is not really what they perceived themselves in th